## Seismograms 101

In this exercise, we will download real seismic data and explore the data in the Fourier space.

In [ ]:
!pip install obspy

In [ ]:
import obspy
import numpy
import scipy

from obspy.clients.fdsn.client import Client 
from scipy.signal import decimate, butter,filtfilt
import matplotlib.pyplot as plt

import scipy.signal as signal

In [ ]:
client = Client("IRIS") # connect to IRIS earthquake service
R1=350 # radius the source
# provide times t1 at origin time and one hour after that 
t1 = obspy.UTCDateTime("2022-12-20") # for example obspy.UTCDateTime("2000-01-01T00:00:00") for 1/1/2000
t2 = t1+3600*24  # origin time + one hour in seconds
cat = client.get_events(starttime=t1, endtime=t2, minmagnitude=6)
# print(cat[0].origins[0])
# Main shock info:
Mag = cat[0].magnitudes[0].mag
print("Magnitude of the earthquale:",Mag)
t0 = cat[0].origins[0].time
print("origin time of the earthquake:",t0)
lat0 = cat[0].origins[0].latitude
lon0 = cat[0].origins[0].longitude
depth0 = cat[0].origins[0].depth/1000 # convert to km
print("Hypocenter of the earthquake:",lat0,lon0,depth0)

In [ ]:
st=client.get_stations(network="UW",station="LEBA",location="*",channel="*Z",\
                       latitude=lat0,longitude=lon0,maxradius=R1/111.25,starttime=t0-1800,endtime=t0+86400,level="response")
print(st)

We see above the available stations in the state of Oregon (still PNSN stations!). We see several channels that illustrate the types of instruments there are in a networks.

Let's just focus on BASIN. We will request all of the station metadata. Obspy return things in complex nested objects to have all of the information. We relaly want just tghe HN* data. We will download data just before and 2 hours after the origin time of the earthquake

In [ ]:
basin=client.get_stations(network="UO",station="BASIN",location="*",channel="HN*",\
                       starttime=t0-1800,endtime=t0+7200,level="response")

print(basin)
print(basin[0][0])  

* Z -> vertical
* N -> North
* E -> East

UO.BASIN..HN* is a strong motion sensor. Some more information here: https://pnsn.org/seismogram/current/basin
 It is set to low gain so that the instrument does not saturate (clip) in case of strong ground motions.

Let's download and plot the data.

In [ ]:
net = basin[0].code
sta = basin[0][0].code
print(net,sta)
chan = "HNZ"
tr=client.get_waveforms(network=net,station=sta,channel=chan,location="*",starttime=t0-200,endtime=t0+600,attach_response=True);
tr.remove_response()
tr.plot() 

The data is in acceleration. Let's integrate to velocity and then displacement

In [ ]:
from scipy.integrate import cumtrapz

acc = tr[0].data
vel = cumtrapz(acc)*tr[0].stats.delta
disp = cumtrapz(vel)*tr[0].stats.delta

fig,ax=plt.subplots(3,1)

ax[0].plot(acc)
ax[1].plot(vel)
ax[2].plot(disp)


Let's do the same analysis but in the frequency domain